# Lausemallide automaatne tuvastamine koostatud andmebaasi põhjal

In [1]:
#impordid
import pandas as pd
import sqlite3
import numpy as np
import csv

# andmebaasi failinimi, kust andmeid loetake
filename = 'data/andmebaasifailid/tasak_obl_verb_subj_obj_nocase_24_03.db'

# andmebaasiga ühenduse loomine
conn = sqlite3.connect(filename)
cursor = conn.cursor()

### Loeb andmebaasist andmed dataframe'idesse

In [2]:
#funktsioonid andmebaasist andmete võtmiseks vastavalt verbile

#siin võetakse välja kõik read, mis käivad otsitava verbi/ühendverbi kohta, va need, kus süntaktilist rolli pole täpsustatud
def koik_fun(verb, yhend=''):
    query = ("SELECT * FROM obl_verb_subj_obj_nocase WHERE verb == '" + verb + "' and verb_compound == '" + yhend + "' and "
             "deprel2 != '' ORDER BY count DESC")
    df_soltuvused = pd.read_sql_query(query, conn)
    return df_soltuvused

#siin võetakse välja andmed lauseliikmete lihtsagedusest ehk read, kus lauseliikmed on samad
def lihtsagedus_fun(verb, yhend=''):
    query = ("SELECT * FROM obl_verb_subj_obj_nocase WHERE verb =='" + verb + "' and verb_compound == '" + yhend + "' and deprel1 == deprel2 "
            "and deprel1 != '' and deprel2 != '' and case1 == case2 ORDER BY count DESC")
    df_lihtsagedus = pd.read_sql_query(query, conn)
    return df_lihtsagedus

### Leiab eraldi failist verbi enda sageduse, arvutab selle põhjal lauseliikmetele ja nende bigrammidele suhtelise esinemissageduse ja jätab ainult need, mille suhteline sagedus on vähemalt 5%

In [3]:
#leiab verbi/ühendverbi enda sagedus korpuses, et sellega pärast suhteline sagedus leida
def verbi_sagedus_fun(verb, yhend=''):
    #leiame verbi enda esinemissageduse korpuses
    esinemised = pd.read_csv('data/tasak_nocase_statistika_24_03.csv')
    if yhend == '': #kui pole ühendverb
        #võtame sealt välja rea, kus ühendverbi osise veerg on tühi 
        verbirida = esinemised.loc[(esinemised['lemma'] == verb) & (esinemised['compound'].isnull())]
    else: #kui on ühendverb
        #otsime rida, mis käib meie peaverbi ja selle ühendiverbiosise kohta
        verbirida = esinemised.loc[(esinemised['lemma'] == verb) & (esinemised['compound'] == yhend)]
    verbi_sagedus = verbirida['occurences'].tolist()[0]
    return verbi_sagedus

#lisab dataframe-i uue suhteliste sageduste veeru
def suhteline_sagedus(dataframe, v_sagedus):
    suhtsagedused = []
    for sagedus in dataframe['count']:
        suhtsagedus = sagedus/v_sagedus*100
        suhtsagedused.append(suhtsagedus)
    dataframe['rel_freq'] = suhtsagedused
    #jätame suhted, mis esinesid rohkem või võrdselt 5% andmestikus
    dataframe_filtered = dataframe.loc[(dataframe['rel_freq'] >= 5)]
    return dataframe_filtered

### Paneb lävendi ületanud lauseliikmed ja nende paarid suuremateks lausemallideks kokku

In [4]:
def lausemall_jarjekorrata(verb, osis=''):
    
    #siin võetakse andmebaasist verbi kohta käivad read välja ja tehakse dataframeideks
    df_koik = koik_fun(verb, yhend=osis)
    df_lihtsagedus = lihtsagedus_fun(verb, yhend=osis)
    
    #siin võetakse välja lauseliikmete sõltuvused ehk sellised read, kus lauseliikmed on erinevad
    df_koondatud = pd.merge(df_koik, df_lihtsagedus, how='outer', indicator=True)
    df_soltuvused = df_koondatud[df_koondatud['_merge'] == 'left_only'].drop(columns=['_merge'])
    
    #leiame verbi enda esinemissageduse korpuses
    verbi_sagedus = verbi_sagedus_fun(verb, yhend=osis)

    #lisame sõltuvuste tabelisse uue suhteliste sageduste veeru, suhteline sagedus on suurem/võrdne kui 5%
    df_filtered = suhteline_sagedus(df_soltuvused, verbi_sagedus)
    
    #lisame lihtsageduste tabelisse uue suhteliste sageduste veeru, suhteline sagedus on suurem/võrdne kui 5%
    df_lihtsagedus_filtered = suhteline_sagedus(df_lihtsagedus, verbi_sagedus)
    
    suhted_dict_tuple = {} #siia tuleb key lauseliige ja value list temaga koosesinevatest lauseliikmete tupleitest
    suhted_dict_pd = {} #siia tuleb key deprel1 lauseliige ja value suhted_dict_pd_vaike
    suhted_dict_pd_nested = {} #siia tuleb key deprel1 lauseliige ja value suhted_dict_pd_vaike

    #siin otsitakse iga lauseliikmega koos esinenud teisi lauseliikmeid ja pannakse sõnastikku
    for ind in df_lihtsagedus_filtered.index: #võtame lihtsageduste kaupa üksikuid lauseliikmeid
        #siia tuleb key deprel2 lauseliige ja value list temaga koosesinevatest lauseliikmete dataframeidest
        suhted_dict_pd_vaike = {} 
        deprel1 = df_lihtsagedus_filtered['deprel1'][ind]
        case1 = df_lihtsagedus_filtered['case1'][ind]
        oblcase1 = df_lihtsagedus_filtered['obl_case1'][ind]
        verbform = df_lihtsagedus_filtered['verbform1'][ind]
        dep1_tuple = (deprel1, case1, oblcase1, verbform)
        
        #otsime read, kus vaatlusealune lauseliige esineb koos teise lauseliikmega
        suhted = df_filtered.loc[(df_filtered["deprel1"] == deprel1) & 
                                (df_filtered["case1"] == case1) &
                                (df_filtered["obl_case1"] == oblcase1) &
                                (df_filtered["verbform1"] == verbform)]
        
        if not suhted.empty: #kui lauseliige esineb teiste lauseliikmetega
            suhte_list = [] #siia salvestame kõik dep1-ga koosesinevate lauseliikmete tuplei-id
            for index, suhe in suhted.iterrows():
                dep2_tuple = (suhe['deprel2'], suhe['case2'], suhe['obl_case2'], suhe['verbform2'])
                suhte_list.append(dep2_tuple) #lisame selle lauseliikmega koosesinevad lauseliikmete tupleid suuremasse listi
                suhted_dict_pd_vaike[dep2_tuple] = suhe
            suhted_dict_tuple[dep1_tuple] = suhte_list #lisame dicti
            suhted_dict_pd[dep1_tuple] = suhted #lisame dicti
            suhted_dict_pd_nested[dep1_tuple] = suhted_dict_pd_vaike #lisame dicti
     
    #siin pannakse need koos esinenud lauseliikmed pikemateks kokku
    lausemallid_yhekordsed = [] #siia tulevad lausemallid listina  
    lausemallid_set = [] #siia tulevad mallid settidena, et saaks korduvad välja visata, aga järjekord sassi ei läheks
    for dep1, values in suhted_dict_tuple.items(): #lähme dep1 kaudu 
        for dep2 in values: #lähme dep2 kaudu
            
            #lisame bigrammi
            bigramm = {verb, osis, dep1, dep2}
            if bigramm not in lausemallid_set: #viskame korduvad välja
                lausemallid_set.append(bigramm) #lisame korduvate kontrollimiseks lausemallide setid listi
                #et lauseliikmete järjekord säiliks, lisame lausemalli ka listina listi
                lausemallid_yhekordsed.append([verb, osis, dep1, dep2]) 
            
            #võtame välja dep2-ga koosesinenud lauseliikmed
            suhted_dep2 = suhted_dict_tuple[dep2]
            #võtame välja dep1-ga koosesinenud lauseliikmed
            suhted_dep1 = suhted_dict_tuple[dep1]
            #leiame, mis lauseliikmed olid mõlemas sagedad
            sage_molemas = [value for value in suhted_dep1 if value in suhted_dep2]
            
            #lisame trigrammid lausemallidesse
            for dep3 in sage_molemas: 
                trigramm = {verb, osis, dep1, dep2, dep3}
                if trigramm not in lausemallid_set:
                    lausemallid_set.append(trigramm)
                    lausemallid_yhekordsed.append([verb, osis, dep1, dep2, dep3])
                
                #leiame tetragrammid
                suhted_dep3 = suhted_dict_tuple[dep3] #võtame dep3-ga koosesinenud lauseliikmed
                #leiame ühisosa dep1, dep2 ja dep3 lauseliikmete vahel
                sage_koigis = [value for value in sage_molemas if value in suhted_dep3]
                for dep4 in sage_koigis: #loopime üle teiste sage_molemas liikmete ehk dep4-de
                    tetragramm = {dep1, verb, osis, dep2, dep3, dep4}
                    if tetragramm not in lausemallid_set:
                        lausemallid_set.append(tetragramm)
                        lausemallid_yhekordsed.append([verb, osis, dep1, dep2, dep3, dep4])
                    
                    #juhul kui pentagramme tahta, aga liigutamisverbidel neid ei olnud
                    #suhted_dep4 = suhted_dict_tuple[dep4] #võtame dep4-ga koosesinenud lauseliikmed
                    #sage_koigis_2 = [value for value in sage_koigis if value in suhted_dep4]
                    #print(sage_koigis_2)
    
    #leiame iga lauseliikme jaoks pikima lausemalli, kus ta esineb
    pikimad_mallid_dict = {}
    for mall in lausemallid_yhekordsed:
        for lauseliige in mall:
            if type(lauseliige) != str: #filtreerime verbi ja ühendverbi osise välja
                #võtame pikima malli
                if lauseliige not in pikimad_mallid_dict or len(mall) > len(pikimad_mallid_dict[lauseliige][0]):
                    pikimad_mallid_dict[lauseliige] = [mall]
                #kui mall on pikimaga ühepikkune, võtame selle ka
                elif len(mall) == len(pikimad_mallid_dict[lauseliige][0]):
                    pikimad_mallid_dict[lauseliige].append(mall)
    
    #eemaldame korduvad lausemallid
    pikimad = set()
    for pikkmallid in pikimad_mallid_dict.values():
        for pikkmall in pikkmallid:
            pikimad.add(tuple(pikkmall))
    
    pikimad = list(pikimad)
    
    #tagastab pikimad mallid
    return pikimad

### Paneb lauseliikmed õigesse järjekorda

In [5]:
#vähendab list listis jama
def flat_mallid(lausemallid):
    flatid = []
    for mall in lausemallid:
        for m in mall:
            flatid.append(m)
    return flatid

#määrame lauseliikmete järjekorra ja sorteerime vastavalt mallid pikkuse järgi
def sorteeri(mallid):
    sorteeritud = []
    for mall in mallid:
        #teeme dummy väärtustega listi, et lauseliikmed õiges järjekorras salvestada
        uus_mall = [mall[0], mall[1], tuple(), tuple(), tuple(), tuple(), tuple(), tuple(), tuple(), tuple(), tuple(), tuple(),
                    tuple(), tuple(), tuple(), tuple(), tuple(), tuple(), tuple(), tuple(), tuple(), tuple(), tuple(), tuple(), 
                    tuple(), tuple()]
        
        #lauseliikmete loendurid, et kui sama süntaktilise funktsiooniga lauseliikmeid esineb rohkem kui üks, 
        #siis need ei overrideiks üksteist
        xcomp_i = 0
        obl_i = 0
        advmod_i = 0
        advcl_i = 0
        ccomp_i =  0
        #paneme malli tähestikujärjekorda, et sama süntaktilise funktsiooniga aga erineva käändega lauseliikmed alati
        #samas järjekorras oleksid
        mall_sorted = sorted(mall[2:], key=lambda x: (x[0], x[1], x[2], x[3]))
        #paneme lauseliikmed õigesse järjekorda
        for liige in mall_sorted:
            if 'nsubj' in liige:
                uus_mall[2] = liige
            if 'obj' in liige:
                uus_mall[3] = liige
            if 'xcomp' in liige:
                uus_mall[4+xcomp_i] = liige
                xcomp_i += 1
            if 'obl' in liige:
                uus_mall[8+obl_i] = liige
                obl_i += 1
            if 'advmod' in liige:
                uus_mall[12+advmod_i] = liige
                advmod_i += 1
            if 'advcl' in liige:
                uus_mall[16+advcl_i] = liige
                advcl_i += 1
            if 'ccomp' in liige:
                uus_mall[20+ccomp_i] = liige
                ccomp_i += 1
        
        #eemaldame tühjad tupleid
        uus_mall_tyhjata = []
        for l in uus_mall:
            if type(l) == str or (type(l) == tuple and l):
                uus_mall_tyhjata.append(l)
        sorteeritud.append(uus_mall_tyhjata)
    
    #sorteerime mallid vastavalt pikkusele
    sorteeritud_pikkus = sorted(sorteeritud, key=len)
    return sorteeritud_pikkus

### Rakendab eelnevaid funktsioone ja kirjutab leitud mallid csv faili

In [6]:
verbid = ['riputama', 'tirima', 'loopima', 'paiskama', 'raputama', 'torkama', 'sättima', 'toppima', 'laduma', 'poetama', 
          'pistma', 'asetama', 'langetama', 'sõidutama', 'paigutama', 'vedama', 'heitma', 'lükkama', 'tõukama', 'seadma',
          'keerama', 'viskama', 'tõmbama', 'tõstma', 'liigutama', 'viima', 'tooma', 'panema']

mallid_suur_yhekordsed = []

#leiab iga verbi lausemallid
for verb in verbid:
    mallid_suur_yhekordsed.append(lausemall_jarjekorrata(verb))
    
#flattenib list listis
flat_mallid_yhekordsed = flat_mallid(mallid_suur_yhekordsed)

#paneb lauseliikmed õigesse järjekorda
sorteeritud = sorteeri(flat_mallid_yhekordsed)

In [7]:
#kirjutame lausemallid lauseliikmete kaupa csv faili
veerud = ['verb', 'compound', 'lauseliige1', 'lauseliige2', 'lauseliige3','lauseliige4']

with open('data/lausemallid/mallid_test.csv', 'w', newline='') as f:
    write = csv.writer(f)
    write.writerow(veerud)
    write.writerows(sorteeritud)

### Leiab kattuvad mallid ja neis esinevad verbid ja kirjutab need csv faili

In [8]:
#leiame mallid, mis verbidel kattuvad
kattuvad = {}
for mall in sorteeritud:
    verb = mall[:1][0]
    lausemall = tuple(mall[2:])
    if lausemall not in kattuvad.keys():
        kattuvad[lausemall] = [verb]
    elif lausemall in kattuvad.keys():
        kattuvad[lausemall].append(verb)

#sorteerime verbide arvu põhjal kahanevalt
kattuvad_sort = sorted(kattuvad.items(), key=lambda x:len(x[1]), reverse=True)

In [9]:
#kirjutame verbiülesed lausemallid koos neis esinevate verbidega csv faili  
veerud = ['lausemall', 'verbid']

with open('data/lausemallid/mallide_kattuvus.csv', 'w', newline='') as f:
    write = csv.writer(f)
    write.writerow(veerud)
    write.writerows(kattuvad_sort)

### Leiab lausemallides esinevad lauseliikmed, mis verbide mallides need olid ja kirjutab csv faili

In [33]:
#leiame, mis lauseliikmed mis verbide lausemallides esinesid
lauseliikmed = {}

for mall in sorteeritud:
    for liige in mall[2:]:
        if liige not in lauseliikmed.keys():
            lauseliikmed[liige] = [mall[0]]
        elif mall[0] not in lauseliikmed[liige]:
            lauseliikmed[liige].append(mall[0])

#sorteerime selle järgi, mis lauseliikmed kõige rohkemate verbidega esinesid
lauseliikmed_sort = sorted(lauseliikmed.items(), key=lambda x:len(x[1]), reverse=True)

In [26]:
#kirjutame verbiülesed lausemallid koos neis esinevate verbidega csv faili  
veerud = ['lauseliige', 'verbid']

with open('data/lausemallid/lauseliikmed.csv', 'w', newline='') as f:
    write = csv.writer(f)
    write.writerow(veerud)
    write.writerows(lauseliikmed_sort)

### Magistritöös seda ei kasutatud, aga kood töötab ka ühendverbidega

In [22]:
#viskas errorit: tirima välja, vedama kaasa, vedama välja
#andmebaasis pole: rinda pistma, edasi laduma, tagasi laduma, tuld otsa pistma (otsa pistma on), pilku heitma, valgust heitma,
#varju heitma, hinge heitma, meelt heitma, pikali heitma, käima lükkama, samme seadma, kahtluse alla seadma, käkki/sitta keerama
#selili keerama, pilku/silma peale viskama, viina viskama, nalja viskama, väravat viskama, hinge tõmbama, kriipsu peale tõmbama,
#haneks tõmbama, võrdusmärki tõmbama, niiti tõmbama, pead tõstma, häält tõstma, alla tõstma, südant liigutama, lillegi liigutama, 
#üles-alla liigutama, edasi-tagasi liigutama, eemale liigutama, ära liigutama, ellu viima, kurssi viima, ilmale tooma, 
#meelde tooma, asemele tooma, paika panema, kirja panema, alust panema, rõhku panema, maksma panema, pahaks panema, 
#tuld otsa panema, punkti panema, riidesse panema, proovile panema, toime panema, hamba alla panema, puusse panema, pidu panema
#aru pähe panema, kätt südamele panema, teele panema, seina äärde panema, imeks panema, mängu panema, mehele panema, 
#paari panema, i-le täppi panema, süüks panema, käima panema, hakkama panema, südamele panema
yhendverbid = {'riputama': ['üles'], 'tirima': ['välja'], 'loopima': ['laiali'], 'paiskama': ['tagasi', 'pärani', 'laiali'], 
               'raputama': ['välja'],
               'toppima': ['alla', 'täis', 'sisse', 'vahele'], 'pistma': ['välja'], 'asetama': ['peale', 'tagasi'], 
               'sõidutama': ['ringi', 'tagasi'], 'paigutama': ['ära', 'ümber'], 
               'vedama': ['kihla', 'alt', 'kohale', 'kinni', 'kaasa', 'välja'], 'heitma': ['ühte', 'ette', 'vahele'],
               'lükkama': ['edasi', 'tagasi', 'ümber', 'laiali', 'kõrvale', 'üles', 'ära', 'kinni', 'välja', 'eemale', 'alla', 
                           'eest'], 
               'tõukama': ['välja', 'ära', 'edasi', 'sisse', 'üles', 'kõrvale', 'maha', 'eemale', 'lahti', 'ümber'],
               'seadma': ['sisse', 'üles', 'valmis', 'ette'], 
               'keerama': ['lukku', 'lahti', 'kokku', 'ära', 'kinni', 'otsa, ringi', 'ümber', 'taha', 'peale', 'üles', 'alla'],
               'viskama': ['välja', 'maha', 'ette', 'alla', 'ära', 'tagasi', 'üle'], 
               'tõmbama': ['sisse', 'ära', 'välja', 'üle', 'ligi', 'alla', 'maha', 'üles', 'ette', 'püsti', 'lahti', 'peale', 
                           'tagasi', 'ringi'], 'tõstma': ['esile', 'üles', 'püsti', 'välja'], 'liigutama': ['edasi'], 
               'viima': ['läbi', 'ära', 'välja', 'edasi', 'kaasa', 'tagasi', 'üle', 'mööda', 'ette', 'alla', 'kokku'], 
               'tooma': ['välja', 'kaasa', 'esile', 'sisse', 'tagasi', 'ära', 'ilmsiks', 'juurde', 'kohale', 'kokku', 'üle'], 
               'panema': ['tähele', 'kinni', 'kokku', 'püsti', 'maha', 'kõrvale', 'vastu', 'ette', 'välja', 'peale', 'sisse',
                         'üles', 'alla', 'külge', 'pihta', 'ümber', 'ära', 'tagasi', 'kaasa', 'täis']}

mallid_yhend_yhekordsed = []

for verb, yhendid in yhendverbid.items():
    for yhend in yhendid:
        mallid_yhend_yhekordsed.append(lausemall_jarjekorrata(verb, yhend))
        
flat_mallid_yhend_yhekordsed = flat_mallid(mallid_yhend_yhekordsed)
sorteeritud_yhend = sorteeri(flat_mallid_yhend_yhekordsed)

In [23]:
#paneb ühendverbide ja lihtverbide mallid kokku
flat_yhekordsed_suur = sorteeritud + sorteeritud_yhend

In [24]:
#kirjutab kõikide verbide mallid ühte csv faili
veerud = ['verb', 'compound', 'lauseliige1', 'lauseliige2', 'lauseliige3','lauseliige4']
    
with open('data/lausemallid/mallid_koikverbid_jarjestatud_pikimad.csv', 'w', newline='') as f:
    write = csv.writer(f)
    write.writerow(veerud)
    write.writerows(flat_yhekordsed_suur)